In [1]:

%load_ext autoreload
%autoreload 2

In [45]:
data = QM9(r"../data/qm9/")

In [53]:
from rdkit import Chem

import pandas as pd
import numpy as np

path = "qm9_smiles.csv"

suppl = Chem.SmilesMolSupplier(path)

df = pd.read_csv(path)

df["target1"] = np.random.randn((len(df)))
df["target2"] = np.random.randn((len(df)))

df.set_index("smiles",inplace=True)

df.to_csv("qm9_target.csv")

## Testing SmilesDataset

In [7]:
from smiles_dataset import SmilesDataset
from smiles_lightning_data_module import SmilesDataModule
from lightning_model import LightningClassicGNN
import pytorch_lightning as pl
from torch_geometric.transforms import distance
from torch_geometric.loader import DataLoader
import os
import torch
# making sure we are as determinstic as possibe
torch.use_deterministic_algorithms(True)
import numpy as np

! rm -rf ../data/test/processed
! rm -rf ../data/test_hydrogen/processed


seed=42
## pytorch lighting takes of seeding everything
pl.seed_everything(seed=seed, workers=True)
# create from csv a pytorch dataset
dataset = SmilesDataset(root=r"../data/test", filename="qm9_target.csv", add_hydrogen=True, transform=distance.Distance())
# from torch dataset, create lightning data module to make sure training splits are always done the same ways
data_module = SmilesDataModule(dataset=dataset, seed=seed)


num_node_features = data_module.num_node_features
num_edge_features= data_module.num_edge_features

gnn_model = LightningClassicGNN(classification=False, output_dim=2, num_node_features=num_node_features, num_edge_features=num_edge_features)

num_epochs=1


# from pytorch_ligthing import loggers
# logger = loggers.WandbLogger()

# default root dir is where the logs and weights are logged
# useful when debugging is limit_train_batches
# by default uses TensorBoardLogger, can be configured 
# Plugins allows us to connect to arbitrary cluster
# can set max_epochs
# can use precision to specify number of bit floating points to reduce memory footprint ()
# can use accumulate_grad_batches to speed-up training too
trainer = pl.Trainer(deterministic=True, auto_lr_find=True, default_root_dir=os.getcwd(), precision="bf16", max_epochs=num_epochs)



Global seed set to 42
Processing...
100%|██████████| 100/100 [00:00<00:00, 105.91it/s]
Done!
Global seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [8]:
# tune to find the learning rate
trainer.tune(gnn_model,datamodule=data_module)

/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LR finder stopped early after 2 steps due to diverging loss.
Failed to compute suggestion for learning rate because there are not enough points. Increase the loop iteration limits or the size of your dataset/dataloader.
Restoring states from the checkpoint path at /home/harold/epfl/chem-ml-repr/code/.lr_find_ef32d5bb-2f32-433c-adc9-58cebf7beeee.ckpt


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7fcce733e460>}

In [5]:

# we can resume from a checkpoint using trainer.fit(ckpth_path="some/path/to/my_checkpoint.ckpt")
trainer.fit(gnn_model, datamodule=data_module)

(5, 5)

In [3]:
from datasets import QM9Dataset
import os

root= "../data/qm9"
! rm -rf ../data/qm9/processed

data = QM9Dataset(root=root, begin_index=100000, end_index=101000)


Processing...


Index(['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0',
       'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom',
       'g298_atom'],
      dtype='object')


6it [00:00, 53.69it/s]

100000 CCC(C)(C)C(C)C=O
100001 CCC(C)(C)C(O)C=O
100002 CCC(C)(C)N(C)C=O
100003 CCC(C)(O)C(C)C=O
100004 CCC(C)(O)C(O)C=O
100005 CC(C)(CO)N(C)C=O
100006 COC(C)(C)C(C)C=O
100007 COC(C)(C)C(O)C=O
100008 CC1(CC1)C(C)(C)CO
100009 CC(C)(CO)C1(C)CO1
100010 CC(C)(CO)C1(CC1)O
100011 CC(O)(CO)C1(C)CC1
100012 CC(O)(CO)C1(C)CN1


23it [00:00, 75.43it/s]

...

995it [00:10, 66.57it/s]

100985 OCC(O)C1(O)CC1O
100986 CC(CO)C1(C)CCC1
100987 CC(CO)C1(C)CCO1
100988 CC(CO)C1(C)COC1
100989 CC(CO)C1(O)CCC1
100990 CC(CO)C1(O)COC1
100991 CC1(CCC1)C(O)CO
100992 CC1(CCO1)C(O)CO
100993 CC1(COC1)C(O)CO
100994 CCC(C)C1(C)CCC1
100995 CCC(C)C1(C)CCO1
100996 CCC(C)C1(C)COC1
100997 CCC(C)C1(O)CCC1
100998 CCC(C)C1(O)COC1


1000it [00:10, 94.68it/s]

100999 CCC(O)C1(C)CCC1
NUM MOLECULES SKIPPED 0, 0.00% of the data



Done!


In [2]:
from datasets import BaceDataset


root= "../data/bace"
! rm -rf ../data/bace/processed

data = BaceDataset(root=root)

Processing...


Index(['Class', 'pIC50'], dtype='object')


0it [00:00, ?it/s]

NUM MOLECULES SKIPPED 0, 0.00% of the data


IndexError: list index out of range

In [1]:
import pandas as pd
import torch


%load_ext autoreload
%autoreload 2

df = pd.read_csv("../data/bace/raw/bace.csv")

BACE_REGRESSION_TASKS = "pIC50"
BACE_CLASSIFICATION_TASKS = "Class"

df = df[["mol", BACE_CLASSIFICATION_TASKS, BACE_REGRESSION_TASKS]]
df.set_index("mol", drop=True, inplace=True)

torch.tensor(df.values)

tensor([[1.0000, 9.1549],
        [1.0000, 8.8539],
        [1.0000, 8.6990],
        ...,
        [0.0000, 2.9531],
        [0.0000, 2.7333],
        [0.0000, 2.5445]], dtype=torch.float64)

array([[1.       , 9.1549015],
       [1.       , 8.8538723],
       [1.       , 8.6989698],
       ...,
       [0.       , 2.9531147],
       [0.       , 2.7332981],
       [0.       , 2.5445461]])

In [7]:
df.Class.factorize()

(array([0, 0, 0, ..., 1, 1, 1]), Int64Index([1, 0], dtype='int64'))